In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/My Drive/BaseDeDatos/ClimateEngineTMax.csv" "/content/"


In [4]:
TMax = pd.read_csv("ClimateEngineTMax.csv",sep=';')


In [5]:
TMax.columns = [None] * len(TMax.columns)
TMax.columns = ['Fecha','Temperatura_Maxima']


In [6]:
TMax['Fecha'] = pd.to_datetime(TMax['Fecha'], format='%Y-%m-%d:%I %p')
TMax['Fecha'] = pd.to_datetime(TMax['Fecha'])
TMax.set_index('Fecha', inplace=True)
TMax = TMax.asfreq('D')

In [7]:
TMax['Temperatura_Maxima'] = TMax['Temperatura_Maxima'].str.replace(',', '.')
TMax['Temperatura_Maxima'] = TMax['Temperatura_Maxima'].astype(float)

In [8]:
TMax.isnull().sum()

Temperatura_Maxima    0
dtype: int64

In [9]:
TMax

,Temperatura_Maxima
Fecha,
2000-01-01,11.45
2000-01-02,13.65
2000-01-03,16.25
2000-01-04,10.95
2000-01-05,9.35
...,...
2022-12-27,21.15
2022-12-28,12.95
2022-12-29,7.45


In [10]:
completo = TMax.copy()

In [11]:
completo

,Temperatura_Maxima
Fecha,
2000-01-01,11.45
2000-01-02,13.65
2000-01-03,16.25
2000-01-04,10.95
2000-01-05,9.35
...,...
2022-12-27,21.15
2022-12-28,12.95
2022-12-29,7.45


In [12]:
fechas_a_eliminar = pd.to_datetime(['2011-08-04','2011-08-05','2011-08-06','2011-08-07','2011-08-08','2011-08-09','2011-08-10',
                                    '2011-09-04','2011-10-04','2011-11-04','2012-11-05','2013-11-06','2014-11-07','2015-11-08',
                                    '2022-06-04','2022-07-04','2022-07-14','2022-07-24','2022-07-25'])

# Utilizar loc para establecer el valor de la columna como NaN en las fechas especificadas
imputar = completo.copy()
imputar.loc[fechas_a_eliminar, 'Temperatura_Maxima'] = None


In [13]:
imputar.isnull().sum()

Temperatura_Maxima    19
dtype: int64

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import tensorflow as tf

In [15]:
# Establecer la semilla para TensorFlow
tf.random.set_seed(123)

imputarTMax = imputar.copy()
# Supongamos que df es tu DataFrame, y 'a_rellenar' es la columna que quieres completar
series = imputarTMax['Temperatura_Maxima']

#Eliminar los valores de nan
series_droppednan = series.dropna()

# Crear secuencias de los datos
sequence_length = 15
sequences = []
for i in range(len(series_droppednan) - sequence_length):
    sequences.append(series_droppednan[i:i+sequence_length].values)
sequences = np.array(sequences)

# Ejemplos y etiquetas
X = sequences[:, :-1]
Y = sequences[:, -1]

# Redimensionar los ejemplos para el LSTM
X = np.expand_dims(X, -1)

# Crear el modelo
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(X.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
model.fit(X, Y, epochs=60, verbose=1)

# Utilizar el modelo para introducir valores faltantes
for i in range(len(series)):
    if pd.isnull(series[i]):
        input = np.expand_dims(np.expand_dims(series[i-sequence_length+1:i].values, -1), 0)
        series[i] = model.predict(input)[0, 0]

# Ahora series debería tener los valores faltantes llenados

Epoch 1/60
262/262 [==============================] - 16s 53ms/step - loss: 17.4971
Epoch 2/60
262/262 [==============================] - 8s 30ms/step - loss: 9.5224
Epoch 3/60
262/262 [==============================] - 8s 31ms/step - loss: 9.3276
Epoch 4/60
262/262 [==============================] - 8s 29ms/step - loss: 9.3560
Epoch 5/60
262/262 [==============================] - 7s 28ms/step - loss: 9.5188
Epoch 6/60
262/262 [==============================] - 10s 36ms/step - loss: 9.3386
Epoch 7/60
262/262 [==============================] - 7s 27ms/step - loss: 9.3046
Epoch 8/60
262/262 [==============================] - 9s 34ms/step - loss: 9.3148
Epoch 9/60
262/262 [==============================] - 7s 25ms/step - loss: 9.3621
Epoch 10/60
262/262 [==============================] - 9s 35ms/step - loss: 9.3406
Epoch 11/60
262/262 [==============================] - 7s 26ms/step - loss: 9.2336
Epoch 12/60
262/262 [==============================] - 9s 35ms/step - loss: 9.2305
Epoch 13/6

In [16]:
# Crea un nuevo DataFrame con la serie
rnn_imputed = pd.DataFrame({'Temperatura_Maxima': series.values}, index=series.index)


In [17]:
imputar.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8401 entries, 2000-01-01 to 2022-12-31
Freq: D
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Temperatura_Maxima  8382 non-null   float64
dtypes: float64(1)
memory usage: 389.3 KB


In [18]:
linear_imputed = imputar.copy()

# Rellenar valores faltantes utilizando interpolate de pandas
linear_imputed['Temperatura_Maxima'] = linear_imputed['Temperatura_Maxima'].interpolate(method='linear')


In [19]:
imputar.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8401 entries, 2000-01-01 to 2022-12-31
Freq: D
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Temperatura_Maxima  8382 non-null   float64
dtypes: float64(1)
memory usage: 389.3 KB


In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
# Identificar dónde 'imputar' originalmente tenía valores NaN para 'Temperatura_Maxima'
nan_indices = imputar['Temperatura_Maxima'].isna()
print(nan_indices)

Fecha
2000-01-01    False
2000-01-02    False
2000-01-03    False
2000-01-04    False
2000-01-05    False
              ...  
2022-12-27    False
2022-12-28    False
2022-12-29    False
2022-12-30    False
2022-12-31    False
Freq: D, Name: Temperatura_Maxima, Length: 8401, dtype: bool


In [22]:
# Asegúrate de que los índices de ambos DataFrame alinean
completo_aligned = completo.loc[nan_indices]
rnn_imputed_aligned = rnn_imputed.loc[nan_indices]
linear_imputed_aligned = linear_imputed.loc[nan_indices]

In [23]:
# Calcular RMSE para la comparación sólo en los puntos donde habían valores nulos originalmente
rmse_rnn = np.sqrt(mean_squared_error(completo_aligned['Temperatura_Maxima'], rnn_imputed_aligned['Temperatura_Maxima']))
rmse_linear = np.sqrt(mean_squared_error(completo_aligned['Temperatura_Maxima'], linear_imputed_aligned['Temperatura_Maxima']))

In [24]:
print(f'RMSE para la imputación usando RNN: {rmse_rnn}')
print(f'RMSE para la imputación usando regresión lineal: {rmse_linear}')

RMSE para la imputación usando RNN: 2.148589636945207
RMSE para la imputación usando regresión lineal: 2.1911396204389413
